In [5]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import google.generativeai as genai
from langdetect import detect
import requests
import json
import os
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configure Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.0-flash')

# Google Safe Browsing API
GOOGLE_SAFE_BROWSING_API_KEY = os.getenv("GOOGLE_SAFE_BROWSING_API_KEY")

def analyze_text(text: str) -> dict:
    lang = detect(text)
    if lang != 'en':
        text = translate_text(text, src=lang, dest='en')
    
    prompt = f"""
        Analyze this message for scam indicators across these categories:

        **1. Urgency & Fear Tactics**
        - Immediate action required (e.g., "account will be closed within 24h")
        - Threats of legal/financial consequences
        - Emergency financial assistance requests

        **2. Financial Requests**
        - Requests for bank details/passwords
        - Cryptocurrency investment offers
        - Unusual payment methods (gift cards, wire transfers)

        **3. Phishing Patterns**
        - Mismatched sender/receiver context
        - Suspicious links (shortened URLs, misspelled domains)
        - Fake login/verification pages

        **4. Fake Offers**
        - Unexpected prizes/lottery wins
        - "Guaranteed" high-return investments
        - Free products with upfront fees

        **5. Impersonation**
        - False claims from banks/government agencies
        - Fake tech support alerts
        - Romance scams with sudden financial needs

        **6. Language Red Flags** (English/Spanish/French)
        - EN: "Verify your account", "Limited time offer"
        - ES: "Oferta exclusiva", "Cuenta bloqueada"
        - FR: "Virement urgent", "Gagnez un prix"

        **Analysis Rules:**
        1. Detect input language (EN/ES/FR)
        2. Score 0-10 scam likelihood (10=definite scam)
        3. List detected patterns with examples
        4. Consider voice note transcription errors

        **Output Format (JSON):**
        {{
        "risk_level": "Safe/Suspicious/Scam",
        "confidence_score": 0-10,
        "detected_patterns": [
            {{
            "category": "Financial Requests",
            "examples_found": ["Request for Bitcoin wallet address"]
            }}
        ],
        "justification": "Concise explanation in user's language"
        }}

        Message: {text}
        """

    response = model.generate_content(prompt)
    result = json.loads(response.text)
    
    if lang != 'en':
        result['justification'] = translate_text(result['justification'], src='en', dest=lang)
    try:
        result = json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "risk_level": "Error",
            "confidence_score": 0,
            "detected_patterns": [],
            "justification": "Failed to parse AI response"
        }
    
    # Add missing translate_text function
    def translate_text(text: str, src: str, dest: str) -> str:
        # Mock translation for testing
        return f"[Translated from {src} to {dest}] {text}"
    
    return result

# Update voice transcription to use proper audio handling
def transcribe_voice(audio_file_bytes: bytes) -> str:
    try:
        response = model.generate_content(
            contents=[
                {"mime_type": "audio/webm", "data": audio_file_bytes},
                "Transcribe this audio message"
            ]
        )
        return response.text
    except Exception as e:
        raise ValueError(f"Transcription failed: {str(e)}")


@app.post("/scan/text")
async def scan_text(message: str = Form(...)):
    try:
        result = analyze_text(message)
        return result
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.post("/scan/voice")
async def scan_voice(file: UploadFile = File(...)):
    try:
        audio_bytes = await file.read()
        text = transcribe_voice(audio_bytes)
        result = analyze_text(text)
        return result
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

# URL scanning remains same as previous implementation
# Update scan_url endpoint
@app.post("/scan/url")
async def scan_url(url: str = Form(...)):
    try:
        response = requests.post(api_url, params=params, json=payload)
        response.raise_for_status()  # Raises HTTPError for 4xx/5xx
        data = response.json()
        risk = "Scam" if data.get("matches") else "Safe"
        return {"url": url, "risk": risk, "details": data.get("matches", [])}
    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={"error": f"URL check failed: {str(e)}"}
        )

# Cell 4: Run the server
import uvicorn

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start in background thread
import threading
thread = threading.Thread(target=run_server, daemon=True)
thread.start()



INFO:     Started server process [2392]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [6]:
# Update test cells
# Cell 5: Test with valid JSON response
response = requests.post(
    "http://localhost:8000/scan/text",
    data={"message": "Hello, this is a safe message"}
)
print("Text scan test:", response.json())

# Cell 6: Test with known safe URL
response = requests.post(
    "http://localhost:8000/scan/url",
    data={"url": "https://www.google.com"}
)
print("URL scan test:", response.json())


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:59030 - "POST /scan/text HTTP/1.1" 500 Internal Server Error
Text scan test: {'error': 'Expecting value: line 1 column 1 (char 0)'}


Task exception was never retrieved
future: <Task finished name='Task-9' coro=<Server.serve() done, defined at c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\server.py:67> exception=SystemExit(1)>
Traceback (most recent call last):
  File "c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\server.py", line 162, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1525, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\

INFO:     127.0.0.1:59037 - "POST /scan/url HTTP/1.1" 500 Internal Server Error
URL scan test: {'error': 'Failed to check URL'}
